In [ ]:
from editor import Editor
from Box2D.b2 import pi

# edit your model by running this cell
# hit 's' to save while editing to model.json

editor = Editor()
editor.load('wheel_arm.json')
editor.run()

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
from IPython.display import clear_output

import pygame
from pygame.locals import (KEYDOWN, K_RIGHT, K_LEFT, K_r, K_s)

from Box2D.b2 import pi

from agent import Agent
from engine import Engine

from time import sleep

# train your model to (hopefully) walk by running this cell
FILE = 'star.json'

TIMESTEPS = 500 # timesteps per episode

# Physics engine and window
LOWER_LIM = -.5 * np.pi
UPPER_LIM = .5 * np.pi
LIMIT = True

ANGLE_RANGE = UPPER_LIM - LOWER_LIM if LIMIT else np.pi*2.0
engine = Engine(caption='Train', width=1280, height=720, gravity=(0, -50), joint_limit=LIMIT, \
               upper_angle=UPPER_LIM, lower_angle=LOWER_LIM, linear_damping=0.0, angular_damping=0.0)

engine.load(FILE)

# Add floor
engine.add_static_body(p=(engine.width/2, engine.height-10), size=(engine.width, 10))

def vis_param(i):
    W = agent.qnet.params[i].eval(agent.qnet.session)
    try:
        fig = plt.imshow(W)
    except:
        pass
    print(np.mean(W))

# Features
ENCODING_SIZE = 32 # buckets for body angles

def features(time_step=0):
    f = []
    """
    for b in engine.world.bodies:
        if b.userData and isinstance(b.userData, dict):
            a = b.angle % ANGLE_RANGE
            a /= ANGLE_RANGE
            a = int(a * ENCODING_SIZE)
            encoding = [0] * ENCODING_SIZE
            encoding[a] = 1.0
            f.extend(encoding)"""
    
    
    for b in engine.world.bodies:
        if b.userData and isinstance(b.userData, dict):
            a = b.angle % ANGLE_RANGE
            #a /= ANGLE_RANGE
            #a = a * 2.0 - 1.0
            f += [np.sin(a), np.cos(a)]
            #f += [b.angularVelocity]
            #f += [b.linearVelocity[0]]
            #f += [b.linearVelocity[1]]
            
    #f += [float(time_step/TIMESTEPS * 2.0 - 1.0)]
    f.append(np.sin(time_step/8.0))
            
    #for j in engine.world.joints:
    #    f.append(j.motorSpeed)
                
    return np.array([f])

# all joints change
s_format = '{:0' + str(len(engine.world.joints)) + 'b}'
def act(action):
    # joint config:
    config = s_format.format(action) # turn into binary rep
    for i, c in enumerate(config):
        engine.world.joints[i].motorSpeed = 20 if c == '1' else -20

# one joint at a time
"""
def act(action):
    #for j in engine.world.joints:
        #j.motorSpeed = 0.0
        
    #for b in engine.world.bodies:
        #if b.userData:
            #b.angularVelocity = 0
            #b.linearVelocity = (0, b.linearVelocity[1])
        
    motorSpeed = 4.0
    cnt = len(engine.world.joints)
    if action > cnt-1:
        motorSpeed *= -1
        action -= cnt
    joint = engine.world.joints[action]
    joint.motorSpeed = motorSpeed"""
    
def body_position():
    avg = np.zeros([2])
    cnt = 0
    for b in engine.world.bodies:
        if b.userData:
            avg += [b.position[0], b.position[1]]
            cnt += 1
    avg /= cnt
    return engine.to_screen(avg)

def set_position(position, zero_vel=True):
    current = engine.to_pybox2d(body_position())
    position = engine.to_pybox2d(position)
    shift = (position[0] - current[0], position[1] - current[1])
    for b in engine.world.bodies:
        if b.userData:
            if zero_vel:
                b.linearVelocity = (0, 0)
                b.angularVelocity = 0
            b.position = (b.position[0] + shift[0], b.position[1] + shift[1])

body_count = len(engine.world.bodies) - 1
joint_count = len(engine.world.joints)
action_count = 2**joint_count

if body_count == 0 or joint_count == 0:
    engine.close()
    raise RuntimeError("Your model doesn't have any physics bodies or joints")

agent = Agent(features=features().shape[1], actions=action_count, hdim=512, gamma=0.95, learning_rate=0.6, \
             max_memory=20000)
agent.eps = .3

rs = []
avg_r = 0.0

speedup = False
running = True
while running:
    
    # train
    if len(agent.M) > 0:
        clear_output(wait=True)
        
        fig = plt.plot(rs, 'b')
        plt.plot([0, len(rs)], [0, 0], 'r')
        plt.show()
        
        if len(rs) > 5000:
            rs = rs[4000:]
            
        if len(agent.M) > 100:
            agent.train(iters=1, batch_size=min(len(agent.M), 1000))
            #vis_param(0)
            #plt.show()
            
    
    # timesteps
    t = 0
    
    engine.clear_all_but_ground()
    engine.load(FILE)
    
    # randomize position
    #for j in engine.world.joints:
    #    j.motorSpeed = np.random.random_sample() * 4.0 - 2.0
        
    engine.step_physics(40)
    for b in engine.world.bodies:
        b.linearVelocity = (0, 0)
        b.angularVelocity = 0
        
    
    last_x = body_position()[0]
    
    s1 = features(t)
    q1 = agent.qnet.ff(s1)
    a1 = agent.sample_action_eps(q1)
    
    reset = False
    while not reset:
        for event in engine.events():
            if engine.quit_event(event):
                running = False
                reset = True
            if event.type == KEYDOWN:
                k = pygame.key.get_pressed()
                if k[K_RIGHT]:
                    agent.eps += .1
                elif k[K_LEFT]:
                    agent.eps -= .1
                elif k[K_r]:
                    agent.reset()
                elif k[K_s]:
                    speedup = not speedup

        # build xp tuple
        xp = {'s1': s1.copy(), 'a1': a1}

        # act a1
        act(a1)
        
        # step physics engine n times
        #for i in range(10):
        #    engine.step_physics(1)
        #    if not speedup:
        #        engine.render()
        #        engine.clock_tick()
        
        engine.step_physics(1)
        
        #for b in engine.world.bodies:
        #    b.linearVelocity = (0, 0)
        #    b.angularVelocity = 0
        
        # reward move right relative
        reward = body_position()[0] - last_x
        reward = min(max(reward, -1), 1)
        #reward *= -1
        
        # reward height
        #reward += (engine.height - body_position()[1])/engine.height
        
        xp['r'] = reward
        
        avg_r += .001 * (reward - avg_r)
        rs.append(avg_r)
                             
        # state2
        s2 = features(t)
        q2 = agent.qnet.ff(s2)
        a2 = agent.sample_action_eps(q2)

        xp['s2'] = s2.copy()
        xp['a2'] = a2

        agent.memorize(xp)
        if t > TIMESTEPS:
            reset = True
        
        if not speedup:
            clear_output(wait=True)
            print('s1: {}\nqs: {}\navg_r: {} eps: {} memories: {} action: {}'\
                  .format(s2, q2.shape, np.round(avg_r, 1), agent.eps, len(agent.M), a2))
            engine.render()
            engine.clock_tick()
            #fig = plt.plot(q2[0,:])
            #plt.show()

        # prepare for next rounds
        s1 = s2
        a1 = a2
        last_x = body_position()[0]
        t += 1

        if last_x > engine.width-50.0 or last_x < 50.0:
            reset = True
            engine.render()
            engine.clock_tick()

engine.close()

In [ ]:
vis_param(2)

In [ ]:
agent.qnet.ff(s2)

In [ ]:
agent.M[0]

In [ ]:
costs = []
for i in range(1000):
    agent.train(iters=1, batch_size=500)
    c = agent.test(batch_size=500)
    costs.append(c)
    if i % 10 == 0:
        clear_output(wait=True)
        fig = plt.plot(costs)
        plt.show()